In [1]:
import os
import getpass

os.environ["LLAMA_CLOUD_API_KEY"] = getpass.getpass("LLamaParse API Key:")

In [2]:
os.environ["GROQ_API_KEY"] = getpass.getpass("GROQ_API_KEY:")

In [ ]:
from llama_index.core import VectorStoreIndex, SimpleDirectoryReader
from llama_index.core import StorageContext
from llama_index.embeddings.huggingface import HuggingFaceEmbedding
from IPython.display import Markdown, display
import chromadb

In [ ]:
from llama_index.vector_stores.chroma import ChromaVectorStore


In [3]:
import nest_asyncio

nest_asyncio.apply()

In [4]:
from llama_parse import LlamaParse

parser = LlamaParse(
    result_type="markdown",
    verbose=True,
    language="en",
    num_workers=1,
    premium_mode = True
)

In [5]:
documents = parser.load_data([r"C:\Users\nisha\Desktop\RAG_Assign\RAG_Assign\iesc111.pdf"])

Parsing files: 100%|██████████| 1/1 [00:20<00:00, 20.28s/it]


In [52]:
len(documents)

14

In [7]:
print(documents[2].text[::])

propagates through the medium. Compression is the region of high pressure and rarefaction is the region of low pressure. Pressure is related to the number of particles of a medium in a given volume. More density of the particles in the medium gives more pressure and vice versa. Thus, propagation of sound can be visualised as propagation of density variations or pressure variations in the medium.

The regions where the coils become closer are called compressions (C) and the regions where the coils are further apart are called rarefactions (R). As we already know, sound propagates in the medium as a series of compressions and rarefactions. Now, we can compare the propagation of disturbance in a slinky with the sound propagation in the medium. These waves are called longitudinal waves. In these waves the individual particles of the medium move in a direction parallel to the direction of propagation of the disturbance. The particles do not move from one place to another but they simply osc

In [8]:
print(documents[3].text[::])

Fig. 11.6(b) represent the density and pressure variations, respectively, as a sound wave propagates in the medium.

Compressions are the regions where particles are crowded together and represented by the upper portion of the curve in Fig. 11.6(c). The peak represents the region of maximum compression. Thus, compressions are regions where density as well as pressure is high. Rarefactions are the regions of low pressure where particles are spread apart and are represented by the valley, that is, the lower portion of the curve in Fig. 11.6(c). A peak is called the crest and a valley is called the trough of a wave.

The distance between two consecutive compressions (C) or two consecutive rarefactions (R) is called the wavelength, as shown in Fig. 11.6(c). The wavelength is usually represented by λ (Greek letter lambda). Its SI unit is metre (m).

| Heinrich Rudolph Hertz |
|-------------------------|
| was born on 22 February 1857 in Hamburg, Germany and educated at the University of Ber

In [35]:
from llama_index.llms.groq import Groq

In [36]:
llm = Groq(model="llama3-70b-8192", api_key="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")
from llama_index.core.llms import ChatMessage

messages = [
    ChatMessage(
        role="system", content="You are a pirate with a colorful personality"
    ),
    ChatMessage(role="user", content="What is your name"),
]
resp = llm.chat(messages)

In [37]:
print(resp)

assistant: Arrr, me hearty! Me name be Captain Zephyr "Blackheart" McSnively, the most feared and infamous pirate to ever sail the Seven Seas! Me reputation precedes me, and me exploits be the stuff o' legend. Me ship, the "Maverick's Revenge", be the fastest and most feared vessel on the high seas, crewed by the scurviest dogs and scallywags to ever hoist the Jolly Roger!

But don't ye be thinkin' I be all about plunderin' and pillagin', matey! I be a pirate o' principle, with a heart o' gold and a sense o' humor as sharp as me trusty cutlass. I be a master o' disguise, a virtuoso o' villainy, and a sultan o' swashbucklin'! So hoist the colors, me hearty, and let's set sail fer a life o' adventure and mayhem on the high seas! Yarrr!


In [46]:
from llama_index.core.node_parser import SentenceSplitter
from llama_index.core.node_parser import MarkdownNodeParser
from llama_index.core.node_parser import MarkdownElementNodeParser


In [13]:


splitter = SentenceSplitter(
    chunk_size=1024,
    chunk_overlap=20,
)
nodes = splitter.get_nodes_from_documents(documents)

nodes

TextNode(id_='8f21af8e-30d5-4be4-a19e-7da79f1160b6', embedding=None, metadata={}, excluded_embed_metadata_keys=[], excluded_llm_metadata_keys=[], relationships={<NodeRelationship.SOURCE: '1'>: RelatedNodeInfo(node_id='60eb069c-96a8-4fc2-9f23-26af4ef65def', node_type=<ObjectType.DOCUMENT: '4'>, metadata={}, hash='b9c97f97060c58d1e6c2fb055eebdda3d3f2d08b384a6265784938b7ef9e2f12')}, text='# Chapter 11\n\n## SOUND\n\nEveryday we hear sounds from various sources like humans, birds, bells, machines, vehicles, televisions, radios etc. Sound is a form of energy which produces a sensation of hearing in our ears. There are also other forms of energy like mechanical energy, light energy, etc. We have talked about mechanical energy in the previous chapters. You have been taught about conservation of energy, which states that we can neither create nor destroy energy. We can just change it from one form to another. When you clap, a sound is produced. Can you produce sound without utilising your ener

In [16]:
len(nodes)

14

In [15]:

splitter1 = MarkdownNodeParser(
    include_metadata= False,
    include_prev_next_rel= True,
)

nodes_markdown = splitter1.get_nodes_from_documents(documents)

nodes_markdown[1]


TextNode(id_='6c52dede-cd03-427c-8f08-eb37983b2681', embedding=None, metadata={}, excluded_embed_metadata_keys=[], excluded_llm_metadata_keys=[], relationships={<NodeRelationship.SOURCE: '1'>: RelatedNodeInfo(node_id='60eb069c-96a8-4fc2-9f23-26af4ef65def', node_type=<ObjectType.DOCUMENT: '4'>, metadata={}, hash='b9c97f97060c58d1e6c2fb055eebdda3d3f2d08b384a6265784938b7ef9e2f12'), <NodeRelationship.NEXT: '3'>: RelatedNodeInfo(node_id='8ffa0169-8dcd-45e9-aa3c-31c8483ff85e', node_type=<ObjectType.TEXT: '1'>, metadata={}, hash='ce8807df3795ea8f4b9b1d252e2877e2060c59110af3e1d34536827d62da7ce1')}, text='Chapter 11', mimetype='text/plain', start_char_idx=2, end_char_idx=12, text_template='{metadata_str}\n\n{content}', metadata_template='{key}: {value}', metadata_seperator='\n')

In [82]:
node_parser = MarkdownElementNodeParser(llm=llm, num_workers=8)

In [83]:
nodes_markdown_element = node_parser.get_nodes_from_documents(documents=documents)

0it [00:00, ?it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]
1it [00:00, ?it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]


In [86]:
nodes_markdown_element[7]

IndexNode(id_='db060258-4878-4705-be5c-386b4978e71f', embedding=None, metadata={'col_schema': 'Column: State\nType: string\nSummary: None\n\nColumn: Substance\nType: string\nSummary: None\n\nColumn: Speed in m/s\nType: number\nSummary: None'}, excluded_embed_metadata_keys=['col_schema'], excluded_llm_metadata_keys=[], relationships={<NodeRelationship.SOURCE: '1'>: RelatedNodeInfo(node_id='7a502ce8-cada-4f01-b621-6b8f4ae55c54', node_type=<ObjectType.DOCUMENT: '4'>, metadata={}, hash='9d0d77c9411a751597c9f7f9be255d1ae84b92311c71dcf545bdc170e8689b54'), <NodeRelationship.PREVIOUS: '2'>: RelatedNodeInfo(node_id='56f63ebd-6025-4439-8286-993512753375', node_type=<ObjectType.TEXT: '1'>, metadata={}, hash='d8f6433e378187647f66887e09562443b2f6efd17af31115e71054d502320695'), <NodeRelationship.NEXT: '3'>: RelatedNodeInfo(node_id='8ea8291c-cc0c-4de4-a8a9-be49d3a6b694', node_type=<ObjectType.TEXT: '1'>, metadata={'table_df': "{' State   ': {0: ' Solids  ', 1: '         ', 2: '         ', 3: '       

In [87]:
base_nodes, objects = node_parser.get_nodes_and_objects(nodes_markdown_element)

In [88]:
index_markdown_element = VectorStoreIndex(nodes=base_nodes+objects)

In [60]:
%pip install llama-index-postprocessor-flag-embedding-reranker git+https://github.com/FlagOpen/FlagEmbedding.git

  Cloning https://github.com/FlagOpen/FlagEmbedding.git to c:\users\nisha\appdata\local\temp\pip-req-build-or150le_
  Resolved https://github.com/FlagOpen/FlagEmbedding.git to commit 1d4a772217981bda5837092d642f48c0bab6516b
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
  Preparing metadata (pyproject.toml): started
  Preparing metadata (pyproject.toml): finished with status 'done'
Note: you may need to restart the kernel to use updated packages.


  Running command git clone --filter=blob:none --quiet https://github.com/FlagOpen/FlagEmbedding.git 'C:\Users\nisha\AppData\Local\Temp\pip-req-build-or150le_'


In [61]:
from llama_index.postprocessor.flag_embedding_reranker import FlagEmbeddingReranker


In [89]:
reranker = FlagEmbeddingReranker(
    top_n=3,
    model="BAAI/bge-reranker-large",
)

In [90]:
markdown_element_query_engine = index_markdown_element.as_query_engine(
    similarity_top_k=5,
    node_postprocessors=[reranker],
    verbose=True
)

In [91]:
eq = "What is the speed of sound in aluminium?"

response_element1 = markdown_element_query_engine.query(eq)
print(response_element1)

Retrieval entering 8ea8291c-cc0c-4de4-a8a9-be49d3a6b694: TextNode
Retrieving from object TextNode with query What is the speed of sound in aluminium?
The speed of sound in aluminium is 6420 m/s.


In [92]:
eq2 = "Two children are at opposite ends of an aluminium rod. One strikes the end of the rod with a stone. Find the ratio of times taken by the sound wave in air and in aluminium to reach the second child."
response_element2 = markdown_element_query_engine.query(eq2)
print(response_element2)

Let's assume the length of the aluminium rod is L. The speed of sound in air is given as 344 m/s. The speed of sound in a medium is a property of that medium, so we can't find the speed of sound in aluminium from the given context. However, we know that the speed of sound in a medium is a constant. Let's call the speed of sound in aluminium 'v_al'. 

The time taken by the sound wave in air to reach the second child is L / 344. The time taken by the sound wave in aluminium to reach the second child is L / v_al. 

The ratio of times taken by the sound wave in air and in aluminium to reach the second child is (L / 344) / (L / v_al) = v_al / 344.


In [19]:
nodes_markdown[1].get_content()


'SOUND\n\nEveryday we hear sounds from various sources like humans, birds, bells, machines, vehicles, televisions, radios etc. Sound is a form of energy which produces a sensation of hearing in our ears. There are also other forms of energy like mechanical energy, light energy, etc. We have talked about mechanical energy in the previous chapters. You have been taught about conservation of energy, which states that we can neither create nor destroy energy. We can just change it from one form to another. When you clap, a sound is produced. Can you produce sound without utilising your energy? Which form of energy did you use to produce sound? In this chapter we are going to learn how sound is produced and how it is transmitted through a medium and received by our ears.'

In [21]:
for i in range(len(nodes_markdown)):
    content = nodes_markdown[i].get_content()
    print(f"The {i+1}th node is \"{content}\"")
    print("-" * 40)


The 1th node is "Chapter 11"
----------------------------------------
The 2th node is "SOUND

Everyday we hear sounds from various sources like humans, birds, bells, machines, vehicles, televisions, radios etc. Sound is a form of energy which produces a sensation of hearing in our ears. There are also other forms of energy like mechanical energy, light energy, etc. We have talked about mechanical energy in the previous chapters. You have been taught about conservation of energy, which states that we can neither create nor destroy energy. We can just change it from one form to another. When you clap, a sound is produced. Can you produce sound without utilising your energy? Which form of energy did you use to produce sound? In this chapter we are going to learn how sound is produced and how it is transmitted through a medium and received by our ears."
----------------------------------------
The 3th node is "11.1 Production of Sound"
----------------------------------------
The 4th node 

In [17]:
len(nodes_markdown)

42

In [23]:
from llama_index.embeddings.huggingface import HuggingFaceEmbedding
embed_model = HuggingFaceEmbedding(model_name="BAAI/bge-small-en-v1.5")


c:\Users\nisha\Desktop\RAG_Assign\RAG_Assign\myenv\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [26]:
from llama_index.core import VectorStoreIndex, SimpleDirectoryReader
from llama_index.core import StorageContext
from llama_index.embeddings.huggingface import HuggingFaceEmbedding
from IPython.display import Markdown, display
import chromadb

In [27]:
from llama_index.vector_stores.chroma import ChromaVectorStore


In [20]:
# create client and a new collection
chroma_client = chromadb.EphemeralClient()
chroma_collection = chroma_client.create_collection("quickstart")


In [24]:
# set up ChromaVectorStore and load in data
vector_store = ChromaVectorStore(chroma_collection=chroma_collection)
storage_context = StorageContext.from_defaults(vector_store=vector_store)
index = VectorStoreIndex.from_documents(
    documents, storage_context=storage_context, embed_model=embed_model
)

In [25]:
db = chromadb.PersistentClient(path="./chroma_db")
chroma_collection = db.get_or_create_collection("quickstart")
vector_store = ChromaVectorStore(chroma_collection=chroma_collection)
storage_context = StorageContext.from_defaults(vector_store=vector_store)

index = VectorStoreIndex.from_documents(
    documents, storage_context=storage_context, embed_model=embed_model
)


In [26]:
# load from disk
db2 = chromadb.PersistentClient(path="./chroma_db")
chroma_collection = db2.get_or_create_collection("quickstart")
vector_store = ChromaVectorStore(chroma_collection=chroma_collection)
index = VectorStoreIndex.from_vector_store(
    vector_store,
    embed_model=embed_model,
)


In [31]:
def index_setup(nodes, embed_model, db_path="./chroma_db"):
    # Create or connect to the persistent ChromaDB
    db = chromadb.PersistentClient(path=db_path)
    chroma_collection = db.get_or_create_collection("quickstart")

    # Create ChromaVectorStore
    vector_store = ChromaVectorStore(chroma_collection=chroma_collection)
    
    # Create storage context and index
    storage_context = StorageContext.from_defaults(vector_store=vector_store)
    index = VectorStoreIndex(
        nodes, storage_context=storage_context, embed_model=embed_model
    )

    return index

In [ ]:
def load_index(embed_model, db_path="./chroma_db"):
    # Load the index from persistent storage
    db = chromadb.PersistentClient(path=db_path)
    chroma_collection = db.get_or_create_collection("quickstart")
    vector_store = ChromaVectorStore(chroma_collection=chroma_collection)

    # Create the index from the vector store
    index = VectorStoreIndex.from_vector_store(
        vector_store=vector_store,
        embed_model=embed_model
    )

    return index

In [32]:
nodes_index = index_setup(nodes,embed_model, db_path="./chroma_db_nodes")


In [45]:
query_engine = nodes_index.as_query_engine(llm=llm
    ,similarity_top_k=3, verbose=True
)

question1 = "Two children are at opposite ends of an aluminium rod. One strikes the end of the rod with a stone. Find the ratio of times taken by the sound wave in air and in aluminium to reach the second child."

response = query_engine.query(question1)
print(response.source_nodes)

[NodeWithScore(node=TextNode(id_='a28db473-cf8f-43bf-b133-6463349a946b', embedding=None, metadata={}, excluded_embed_metadata_keys=[], excluded_llm_metadata_keys=[], relationships={<NodeRelationship.SOURCE: '1'>: RelatedNodeInfo(node_id='bd4b4bb2-802d-4396-a3eb-d940502f730f', node_type=<ObjectType.DOCUMENT: '4'>, metadata={}, hash='638890aae915b4f37b49d9edc8644d54bd7be175b28a333d8c84421a25ff2503')}, text='• Sound properties such as pitch, loudness and quality are determined by the corresponding wave properties.\n\n• Loudness is a physiological response of the ear to the intensity of sound.\n\n• The amount of sound energy passing each second through unit area is called the intensity of sound.\n\n• The audible range of hearing for average human beings is in the frequency range of 20 Hz – 20 kHz.\n\n• Sound waves with frequencies below the audible range are termed "infrasonic" and those above the audible range are termed "ultrasonic".\n\n• Ultrasound has many medical and industrial applic

In [ ]:
# load from disk , nodes 
db2 = chromadb.PersistentClient(path="./chroma_db")
chroma_collection = db2.get_or_create_collection("quickstart")
vector_store = ChromaVectorStore(chroma_collection=chroma_collection)
index = VectorStoreIndex.from_vector_store(
    vector_store,
    embed_model=embed_model,
)

In [40]:
from llama_index.core import Settings


In [41]:
Settings.llm = llm
Settings.embed_model = embed_model

In [29]:
recursive_query_engine = index.as_query_engine(llm=llm
    ,similarity_top_k=5, verbose=True
)

In [30]:
query = "Why is sound wave called a longitudinal wave?"
response = recursive_query_engine.query(query)
print(response)

Sound waves are called longitudinal waves because the individual particles of the medium move in a direction parallel to the direction of propagation of the disturbance, oscillating back and forth about their position of rest.


In [32]:
print(documents[-2].text[::])

• Sound properties such as pitch, loudness and quality are determined by the corresponding wave properties.

• Loudness is a physiological response of the ear to the intensity of sound.

• The amount of sound energy passing each second through unit area is called the intensity of sound.

• The audible range of hearing for average human beings is in the frequency range of 20 Hz – 20 kHz.

• Sound waves with frequencies below the audible range are termed "infrasonic" and those above the audible range are termed "ultrasonic".

• Ultrasound has many medical and industrial applications.

## Exercises

1. What is sound and how is it produced?

2. Describe with the help of a diagram, how compressions and rarefactions are produced in air near a source of sound.

3. Why is sound wave called a longitudinal wave?

4. Which characteristic of the sound helps you to identify your friend by his voice while sitting with others in a dark room?

5. Flash and thunder are produced simultaneously. But thun

In [43]:
print(documents[-2].text[::])

• Sound properties such as pitch, loudness and quality are determined by the corresponding wave properties.

• Loudness is a physiological response of the ear to the intensity of sound.

• The amount of sound energy passing each second through unit area is called the intensity of sound.

• The audible range of hearing for average human beings is in the frequency range of 20 Hz – 20 kHz.

• Sound waves with frequencies below the audible range are termed "infrasonic" and those above the audible range are termed "ultrasonic".

• Ultrasound has many medical and industrial applications.

## Exercises

1. What is sound and how is it produced?

2. Describe with the help of a diagram, how compressions and rarefactions are produced in air near a source of sound.

3. Why is sound wave called a longitudinal wave?

4. Which characteristic of the sound helps you to identify your friend by his voice while sitting with others in a dark room?

5. Flash and thunder are produced simultaneously. But thun

In [45]:
questions = {
1 : "What is sound and how is it produced?",

2: "Describe with the help of a diagram, how compressions and rarefactions are produced in air near a source of sound.",

3: "Why is sound wave called a longitudinal wave?",

4: "Which characteristic of the sound helps you to identify your friend by his voice while sitting with others in a dark room?",

5: "Flash and thunder are produced simultaneously. But thunder is heard a few seconds after the flash is seen, why?",

6: "A person has a hearing range from 20 Hz to 20 kHz. What are the typical wavelengths of sound waves in air corresponding to these two frequencies? Take the speed of sound in air as 344 m s⁻¹.",

7: "Two children are at opposite ends of an aluminium rod. One strikes the end of the rod with a stone. Find the ratio of times taken by the sound wave in air and in aluminium to reach the second child.",

8: "The frequency of a source of sound is 100 Hz. How many times does it vibrate in a minute?",

9: "Does sound follow the same laws of reflection as light does? Explain.",

10: "When a sound is reflected from a distant object, an echo is produced. Let the distance between the reflecting surface and the source of sound production remains the same. Do you hear echo sound on a hotter day?",

11: "Give two practical applications of reflection of sound waves.",

12: "A stone is dropped from the top of a tower 500 m high into a pond of water at the base of the tower. When is the splash heard at the top? Given, g = 10 m s⁻² and speed of sound = 340 m s⁻¹.",

13: "A sound wave travels at a speed of 339 m s⁻¹. If its wavelength is 1.5 cm, what is the frequency of the wave? Will it be audible?",

14: "What is reverberation? How can it be reduced?",

15: "What is loudness of sound? What factors does it depend on?",

16: "How is ultrasound used for cleaning?",

17: "Explain how defects in a metal block can be detected using ultrasound."
}


In [38]:
answers = []

for value in questions.values(): 
    repo = recursive_query_engine.query(value)
    answers.append(repo)

In [42]:
print(answers[6])

To find the frequency of the wave, we can use the formula: speed = wavelength × frequency. Rearranging the formula to solve for frequency, we get: frequency = speed / wavelength. Plugging in the given values, we get: frequency = 339 m/s / 0.015 m = 22600 Hz.

Since the frequency of the wave is 22600 Hz, which is greater than 20000 Hz, it will not be audible to the human ear. The audible range of hearing for average human beings is in the frequency range of 20 Hz – 20 kHz.


In [48]:
questions.items()

dict_items([(1, 'What is sound and how is it produced?'), (2, 'Describe with the help of a diagram, how compressions and rarefactions are produced in air near a source of sound.'), (3, 'Why is sound wave called a longitudinal wave?'), (4, 'Which characteristic of the sound helps you to identify your friend by his voice while sitting with others in a dark room?'), (5, 'Flash and thunder are produced simultaneously. But thunder is heard a few seconds after the flash is seen, why?'), (6, 'A person has a hearing range from 20 Hz to 20 kHz. What are the typical wavelengths of sound waves in air corresponding to these two frequencies? Take the speed of sound in air as 344 m s⁻¹.'), (7, 'Two children are at opposite ends of an aluminium rod. One strikes the end of the rod with a stone. Find the ratio of times taken by the sound wave in air and in aluminium to reach the second child.'), (8, 'The frequency of a source of sound is 100 Hz. How many times does it vibrate in a minute?'), (9, 'Does 

In [51]:
ans = {}
for key, value in questions.items():
    ans[key] = recursive_query_engine.query(value)

In [63]:
print(ans[14])

Reverberation is the persistence of sound due to repeated reflections from surfaces, resulting in a prolonged sound that can be heard even after the original sound has stopped. To reduce reverberation, the roof and walls of a hall or auditorium can be covered with sound-absorbent materials like compressed fibreboard, rough plaster, or draperies. Additionally, selecting seat materials with sound-absorbing properties can also help reduce reverberation.
